In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.preprocessing import label_binarize

def geradataset(tamanho=20, centro=2):
    X, y = make_blobs(n_samples=tamanho, centers=centro, center_box=(0,1.0), cluster_std=0.05)
    return X, y

def plotadataset(X, y):
    plt.xlabel("X1")
    plt.ylabel("X2")
    for k in set(y):
        plt.plot(X[:,0][y==k],X[:,1][y==k],"o", alpha=0.3)


def plotahiperplano(vetor, bias=0, xmin=0, xmax=1):
    xs = np.linspace(xmin, xmax, num=2)
    ys = (-vetor[0] / vetor[1]) * xs - bias / vetor[1]
    plt.plot(xs, ys)
    

X, y = geradataset(100, 4)
plotadataset(X, y)
plt.show()
ybin = label_binarize(y, classes=list(set(y)))*2 - 1
# ybin

## Função de Custo

In [ ]:
class CustoPerceptron():
    @staticmethod
    def erro(y, ypred):
        return y - ypred
    @staticmethod
    def custo(y, ypred):
        return np.sum(CustoPerceptron.erro(y, ypred)**2)
    @staticmethod
    def gradiente(y, ypred, X):
        return np.matmul(X.T ,CustoPerceptron.erro(y, ypred))

## Perceptron

In [ ]:
from sklearn.base import BaseEstimator, ClassifierMixin

class Perceptron(BaseEstimator, ClassifierMixin):
    
    def __init__(self, custo=CustoPerceptron(), alpha=0.1):
        self.w = None
        self.activation = lambda a: (a >= 0)*2-1
        self.alpha = alpha
        self.maxiter = 200
        self.custo = custo

    @staticmethod
    def includebias(X):
        bias = np.ones((X.shape[0],1))
        Xb = np.concatenate((bias, X), axis=1)
        return Xb

    def fit(self, X ,y):
        Xb = Perceptron.includebias(X)
        self.labels = list(set(y))
        if len(self.labels) > 2:
            y = label_binarize(y, classes=self.labels)*2-1
            self.w = np.random.uniform(-1, 1, size=(Xb.shape[1], y.shape[1]))
        else:
            self.w = np.random.uniform(-1, 1, size=Xb.shape[1])
        for _ in range(self.maxiter):
            a = self.preactivate(X)
            ypred = self.activation(a)
            custo = self.custo.custo(y, ypred)
            if custo == 0:
                break
            self.w = self.w + self.alpha * self.custo.gradiente(y, ypred, Xb)
    
    def preactivate(self, X):
        Xb = Perceptron.includebias(X)
        return np.matmul(Xb, self.w)

    def predict(self, X):
        a = self.preactivate(X)
        if len(self.w.shape) > 1:
            idx = np.argmax(a, axis=1)
            ypred = np.array([self.labels[i] for i in idx])
        else:
            ypred = self.activation(a)
        return ypred

In [ ]:
perceptron = Perceptron()
perceptron.fit(X, y)
ypred = perceptron.predict(X)
ypred
print(sum(y == ypred)/len(y))
print(perceptron.w[1:], perceptron.w[0])

plotadataset(X, y)
for i in range(len(set(y))):
    plotahiperplano(perceptron.w[1:, i], perceptron.w[0, i], min(X[:,0]), max(X[:,0]))